In [1]:
# import subprocess


# command = """
# lm_eval --model steered --model_args csv_path=/home/cs29824/matthew/sae_jailbreak_unlearning/models/dog_steer.csv --tasks mmlu_abstract_algebra --batch_size auto --output_path .        --device cuda:1
# """

# # Run the command in the shell
# result = subprocess.run(command, capture_output=True, text=True, shell=True)

# # Return the output or error
# print(result.stdout)  # or result.stderr for error messages


In [6]:
topics = ['wmdp_bio',
          'mmlu_high_school_us_history',
          'mmlu_high_school_geography',
          'mmlu_human_aging']
topics = ",".join(topics)
topics

'wmdp_bio,mmlu_high_school_us_history,mmlu_high_school_geography,mmlu_human_aging'

In [5]:
import os
import subprocess
from tqdm import tqdm

# Specify the folder containing CSV files
folder_path = "/home/cs29824/andre/sae_jailbreak_unlearning/steer_dfs_final/steer_dfs_final"
output_path = "/home/cs29824/andre/sae_jailbreak_unlearning/results/steer_eval"
# Loop through each file in the folder
for filename in tqdm(os.listdir(folder_path)):
    print(filename)
    # Check if the file is a CSV file
    if filename.endswith(".csv"):
        # Construct the full path to the CSV file
        csv_file_path = os.path.join(folder_path, filename)

        # Construct the command with the specific CSV file
        command = f"""lm_eval --model steered --model_args csv_path={csv_file_path} --tasks {topics} --batch_size auto --output_path {output_path}  --device cuda:1
"""
        # Run the command and capture output
        # Run the command in the shell
        result = subprocess.run(command, capture_output=True, text=True, shell=True)


        # Print the result for each CSV file
        print(f"Results for {filename}:")
        print(f"Stdout: {result.stdout}")
        print(f"Stderr: {result.stderr}")
        print(f"Return code: {result.returncode}")
        print("-" * 50)


  0%|          | 0/60 [00:00<?, ?it/s]

top_20_coef_-100_method_proportional_steer.csv


  2%|▏         | 1/60 [05:19<5:14:37, 319.95s/it]

Results for top_20_coef_-100_method_proportional_steer.csv:
Stdout: Loaded pretrained model gemma-2-2b into HookedTransformer
Moving model to device:  cuda:1
Passed argument batch_size = auto:1. Detecting largest batch size
Determined largest batch size: 2
steered (csv_path=/home/cs29824/andre/sae_jailbreak_unlearning/steer_dfs_final/steer_dfs_final/top_20_coef_-100_method_proportional_steer.csv), gen_kwargs: (None), limit: None, num_fewshot: None, batch_size: auto (2)
|        Tasks         |Version|Filter|n-shot|Metric|   |Value |   |Stderr|
|----------------------|------:|------|-----:|------|---|-----:|---|-----:|
|high_school_geography |      1|none  |     0|acc   |↑  |0.1768|±  |0.0272|
|high_school_us_history|      1|none  |     0|acc   |↑  |0.2500|±  |0.0304|
|human_aging           |      1|none  |     0|acc   |↑  |0.3139|±  |0.0311|
|wmdp_bio              |      1|none  |     0|acc   |↑  |0.2467|±  |0.0121|


Stderr: 2025-01-21:08:04:37,863 INFO     [__main__.py:284] Verbosity

In [47]:
!lm-eval --tasks 

usage: lm-eval [-h] [--model MODEL] [--tasks task1,task2]
               [--model_args MODEL_ARGS] [--num_fewshot N]
               [--batch_size auto|auto:N|N] [--max_batch_size N]
               [--device DEVICE] [--output_path DIR|DIR/file.json]
               [--limit N|0<N<1] [--use_cache DIR]
               [--cache_requests {true,refresh,delete}] [--check_integrity]
               [--write_out] [--log_samples]
               [--system_instruction SYSTEM_INSTRUCTION]
               [--apply_chat_template [APPLY_CHAT_TEMPLATE]]
               [--fewshot_as_multiturn] [--show_config] [--include_path DIR]
               [--gen_kwargs GEN_KWARGS]
               [--verbosity CRITICAL|ERROR|WARNING|INFO|DEBUG]
               [--wandb_args WANDB_ARGS] [--hf_hub_log_args HF_HUB_LOG_ARGS]
               [--predict_only] [--seed SEED] [--trust_remote_code]
               [--confirm_run_unsafe_code]
lm-eval: error: argument --tasks/-t: expected one argument
